In [4]:
import duckdb
from pathlib import Path

# Conectar a DuckDB
conn = duckdb.connect(":memory:")

# Instalar extensión de PostgreSQL
print("Instalando extensión de PostgreSQL...")
conn.execute("INSTALL postgres")
conn.execute("LOAD postgres")
print("✓ Extensión cargada\n")

# Ruta de archivos curados
curated_path = Path("../data/curated/")

print("=" * 60)
print("ETL: Parquet → PostgreSQL con DuckDB")
print("=" * 60 + "\n")

# 1. Conectar a PostgreSQL y limpiar TODO
print("Conectando a PostgreSQL...")
pg_conn = "dbname=analytics user=user password=pass host=localhost port=5432"
conn.execute(f"ATTACH '{pg_conn}' AS pg (TYPE POSTGRES)")
print("✓ Conectado\n")

print("🗑️  Eliminando TODAS las tablas y constraints...")
conn.execute("DROP TABLE IF EXISTS pg.sales CASCADE")
conn.execute("DROP TABLE IF EXISTS pg.products CASCADE")
conn.execute("DROP TABLE IF EXISTS pg.customers CASCADE")
conn.execute("DROP TABLE IF EXISTS pg.cities CASCADE")
conn.execute("DROP TABLE IF EXISTS pg.customer_types CASCADE")
conn.execute("DROP TABLE IF EXISTS pg.product_types CASCADE")
conn.execute("DROP TABLE IF EXISTS pg.sale_types CASCADE")
print("✓ PostgreSQL limpio\n")

# 2. Cargar todos los Parquet en DuckDB como tablas temporales
print("Cargando archivos Parquet en DuckDB...\n")

conn.execute(f"CREATE TABLE cities AS SELECT * FROM read_parquet('{(curated_path / 'cities.parquet').as_posix()}')")
count = conn.execute("SELECT COUNT(*) FROM cities").fetchone()[0]
print(f"✓ cities            : {count:,} filas")

conn.execute(f"CREATE TABLE customer_types AS SELECT * FROM read_parquet('{(curated_path / 'customer_types.parquet').as_posix()}')")
count = conn.execute("SELECT COUNT(*) FROM customer_types").fetchone()[0]
print(f"✓ customer_types    : {count:,} filas")

conn.execute(f"CREATE TABLE product_types AS SELECT * FROM read_parquet('{(curated_path / 'product_types.parquet').as_posix()}')")
count = conn.execute("SELECT COUNT(*) FROM product_types").fetchone()[0]
print(f"✓ product_types     : {count:,} filas")

conn.execute(f"CREATE TABLE sale_types AS SELECT * FROM read_parquet('{(curated_path / 'sale_types.parquet').as_posix()}')")
count = conn.execute("SELECT COUNT(*) FROM sale_types").fetchone()[0]
print(f"✓ sale_types        : {count:,} filas")

conn.execute(f"CREATE TABLE customers AS SELECT * FROM read_parquet('{(curated_path / 'customers.parquet').as_posix()}')")
count = conn.execute("SELECT COUNT(*) FROM customers").fetchone()[0]
print(f"✓ customers         : {count:,} filas")

conn.execute(f"CREATE TABLE products AS SELECT * FROM read_parquet('{(curated_path / 'products.parquet').as_posix()}')")
count = conn.execute("SELECT COUNT(*) FROM products").fetchone()[0]
print(f"✓ products          : {count:,} filas")

conn.execute(f"CREATE TABLE sales AS SELECT * FROM read_parquet('{(curated_path / 'sales.parquet').as_posix()}')")
count = conn.execute("SELECT COUNT(*) FROM sales").fetchone()[0]
print(f"✓ sales             : {count:,} filas")

# 3. Transferir datos de DuckDB a PostgreSQL
print("\nTransfiriendo datos a PostgreSQL...\n")

# Orden correcto para respetar foreign keys
transfer_tables = [
    "cities",
    "customer_types", 
    "product_types",
    "sale_types",
    "customers",
    "products",
    "sales"
]

for table in transfer_tables:
    conn.execute(f"CREATE TABLE pg.{table} AS SELECT * FROM {table}")
    count = conn.execute(f"SELECT COUNT(*) FROM pg.{table}").fetchone()[0]
    print(f"✓ {table:20s}: {count:,} filas → PostgreSQL")

print("\n" + "=" * 60)
print("✓ Datos transferidos exitosamente")
print("=" * 60)

conn.close()

Instalando extensión de PostgreSQL...
✓ Extensión cargada

ETL: Parquet → PostgreSQL con DuckDB

Conectando a PostgreSQL...
✓ Conectado

🗑️  Eliminando TODAS las tablas y constraints...
✓ PostgreSQL limpio

Cargando archivos Parquet en DuckDB...

✓ cities            : 33 filas
✓ customer_types    : 4 filas
✓ product_types     : 6 filas
✓ sale_types        : 4 filas
✓ customers         : 132 filas
✓ products          : 72 filas
✓ Extensión cargada

ETL: Parquet → PostgreSQL con DuckDB

Conectando a PostgreSQL...
✓ Conectado

🗑️  Eliminando TODAS las tablas y constraints...
✓ PostgreSQL limpio

Cargando archivos Parquet en DuckDB...

✓ cities            : 33 filas
✓ customer_types    : 4 filas
✓ product_types     : 6 filas
✓ sale_types        : 4 filas
✓ customers         : 132 filas
✓ products          : 72 filas
✓ sales             : 7,431,930 filas

Transfiriendo datos a PostgreSQL...

✓ cities              : 33 filas → PostgreSQL
✓ customer_types      : 4 filas → PostgreSQL
✓ product

In [5]:
import psycopg2

print("=" * 60)
print("Creando relaciones (Primary Keys y Foreign Keys)")
print("=" * 60 + "\n")

# Conectar a PostgreSQL
print("Conectando a PostgreSQL...")
pg_conn = psycopg2.connect(
    dbname="analytics",
    user="user",
    password="pass",
    host="localhost",
    port=5432
)
pg_conn.autocommit = False
cursor = pg_conn.cursor()
print("✓ Conectado\n")

try:
    # 1. Agregar Primary Keys
    print("Agregando Primary Keys...\n")
    
    cursor.execute("ALTER TABLE cities ADD PRIMARY KEY (id_city)")
    print("✓ cities(id_city)")
    
    cursor.execute("ALTER TABLE customer_types ADD PRIMARY KEY (id_type_customer)")
    print("✓ customer_types(id_type_customer)")
    
    cursor.execute("ALTER TABLE product_types ADD PRIMARY KEY (id_type_product)")
    print("✓ product_types(id_type_product)")
    
    cursor.execute("ALTER TABLE sale_types ADD PRIMARY KEY (id_type_sale)")
    print("✓ sale_types(id_type_sale)")
    
    cursor.execute("ALTER TABLE customers ADD PRIMARY KEY (id_customer)")
    print("✓ customers(id_customer)")
    
    cursor.execute("ALTER TABLE products ADD PRIMARY KEY (id_product)")
    print("✓ products(id_product)")
    
    cursor.execute("ALTER TABLE sales ADD PRIMARY KEY (id_sale)")
    print("✓ sales(id_sale)")
    
    pg_conn.commit()
    
    # 2. Agregar Foreign Keys
    print("\nAgregando Foreign Keys...\n")
    
    # Customers → Cities, Customer Types
    cursor.execute("""
        ALTER TABLE customers 
        ADD CONSTRAINT fk_customers_city 
        FOREIGN KEY (id_city) REFERENCES cities(id_city)
    """)
    print("✓ customers → cities")
    
    cursor.execute("""
        ALTER TABLE customers 
        ADD CONSTRAINT fk_customers_type 
        FOREIGN KEY (id_type_customer) REFERENCES customer_types(id_type_customer)
    """)
    print("✓ customers → customer_types")
    
    # Products → Product Types
    cursor.execute("""
        ALTER TABLE products 
        ADD CONSTRAINT fk_products_type 
        FOREIGN KEY (id_type_product) REFERENCES product_types(id_type_product)
    """)
    print("✓ products → product_types")
    
    # Sales → Products, Sale Types, Customers
    cursor.execute("""
        ALTER TABLE sales 
        ADD CONSTRAINT fk_sales_product 
        FOREIGN KEY (id_product) REFERENCES products(id_product)
    """)
    print("✓ sales → products")
    
    cursor.execute("""
        ALTER TABLE sales 
        ADD CONSTRAINT fk_sales_type 
        FOREIGN KEY (id_type_sale) REFERENCES sale_types(id_type_sale)
    """)
    print("✓ sales → sale_types")
    
    cursor.execute("""
        ALTER TABLE sales 
        ADD CONSTRAINT fk_sales_customer 
        FOREIGN KEY (id_customer) REFERENCES customers(id_customer)
    """)
    print("✓ sales → customers")
    
    pg_conn.commit()
    
    print("\n" + "=" * 60)
    print("✓ Todas las relaciones creadas exitosamente")
    print("=" * 60)
    
except Exception as e:
    pg_conn.rollback()
    print(f"\n✗ Error: {e}")
    raise
finally:
    cursor.close()
    pg_conn.close()

Creando relaciones (Primary Keys y Foreign Keys)

Conectando a PostgreSQL...
✓ Conectado

Agregando Primary Keys...

✓ cities(id_city)
✓ customer_types(id_type_customer)
✓ product_types(id_type_product)
✓ sale_types(id_type_sale)
✓ customers(id_customer)
✓ products(id_product)
✓ sales(id_sale)

Agregando Foreign Keys...

✓ customers → cities
✓ customers → customer_types
✓ products → product_types
✓ sales(id_sale)

Agregando Foreign Keys...

✓ customers → cities
✓ customers → customer_types
✓ products → product_types
✓ sales → products
✓ sales → products
✓ sales → sale_types
✓ sales → sale_types
✓ sales → customers

✓ Todas las relaciones creadas exitosamente
✓ sales → customers

✓ Todas las relaciones creadas exitosamente
